# corona to mongoDB

In [1]:
import pyarrow
import requests
import xmltodict
import time
import pandas as pd
import re
from datetime import datetime
import warnings
warnings.filterwarnings('ignore')

import pyspark
from pymongo import MongoClient
from pyspark.conf import SparkConf
from pyspark.sql import SparkSession
from pyspark.sql.types import StructType,StructField, StringType, IntegerType
import pyspark.sql.functions as f

In [30]:
# spark 연결
spark = SparkSession \
    .builder \
    .appName("coronaAPI") \
    .config("spark.mongodb.input.uri", "mongodb://localhost:27017") \
    .config("spark.mongodb.input.database","ojo_db") \
    .config("spark.mongodb.input.collection", "coronaAPI") \
    .config("packages org.mongodb.spark:mongo-spark-connector_2.12:3.0.1").getOrCreate()
sc =spark.sparkContext

IndentationError: unexpected indent (<ipython-input-30-4e831f0f6b43>, line 3)

### 코로나 지역별 확진자 + 단계

In [2]:
#spark = SparkSession.builder.appName("example-pyspark-read-and-write").getOrCreate()

disp = spark.read.parquet("hdfs://localhost:9000/data/coronaAPI_2021-09-29")
disp_st = spark.read.parquet("hdfs://localhost:9000/data/coronaStage_2021-09-29")

df = disp.select('날짜','area','확진자수')
df = df.withColumn('date',f.to_date(df['날짜']))
df.date.cast(StringType())
df = df.drop('날짜')

In [3]:
df.show()

+----+--------+----------+
|area|확진자수|      date|
+----+--------+----------+
|제주|    2860|2021-09-29|
|경남|   11393|2021-09-29|
|경북|    8271|2021-09-29|
|전남|    3060|2021-09-29|
|전북|    4480|2021-09-29|
|충남|    9073|2021-09-29|
|충북|    6284|2021-09-29|
|강원|    6405|2021-09-29|
|경기|   89246|2021-09-29|
|세종|    1225|2021-09-29|
|울산|    5138|2021-09-29|
|대전|    6949|2021-09-29|
|광주|    4957|2021-09-29|
|인천|   15654|2021-09-29|
|대구|   15426|2021-09-29|
|부산|   12672|2021-09-29|
|서울|   99550|2021-09-29|
+----+--------+----------+



In [4]:
disp_st = disp_st.drop('__index_level_0__')
disp_st.show()

+----+------+-----------------------------+
|Area| Stage|                  Description|
+----+------+-----------------------------+
|서울|4 단계|  - 서울 전지역 4단계 (21....|
|부산|3 단계|  - 부산 전지역 3단계 (21....|
|대구|3 단계|  - 대구 전지역 3단계 (21....|
|인천|4 단계| - 인천 일부지역 4단계 (21...|
|광주|3 단계|  - 광주 전지역 3단계 (21....|
|대전|3 단계|  - 대전 전지역 3단계 (21....|
|울산|3 단계|  - 울산 전지역 3단계 (21....|
|세종|3 단계|  - 세종 전지역 3단계 (21....|
|경기|4 단계|  - 경기 전지역 4단계 (21....|
|강원|3 단계|- 춘천시, 원주시, 강릉시, ...|
|충북|3 단계|  - 충북 전지역 3단계 (21....|
|충남|3 단계|  - 충남 전지역 3단계 (21....|
|전북|3 단계| - 전북 일부지역 3단계 (21...|
|전남|3 단계|  - 전남 전지역 3단계 (21....|
|경북|3 단계| - 경북 일부지역 3단계 (21...|
|경남|3 단계|  - 경남 전지역 3단계 (21....|
|제주|3 단계|  - 제주 전지역 3단계 (21....|
+----+------+-----------------------------+



In [27]:
df = df.withColumnRenamed("area", "지역")
df = df.withColumnRenamed("date", "날짜")
disp_st = disp_st.withColumnRenamed("Area", "지역")
disp_st = disp_st.withColumnRenamed("Stage", "거리두기단계")
disp_st = disp_st.withColumnRenamed("Description", "상세내용")

In [28]:
df_corona = df.join(disp_st, on=['지역'], how='left_outer')

In [32]:
df_corona.show()

+----+--------+----------+------------+-----------------------------+
|지역|확진자수|      날짜|거리두기단계|                     상세내용|
+----+--------+----------+------------+-----------------------------+
|제주|    2860|2021-09-29|      3 단계|  - 제주 전지역 3단계 (21....|
|경남|   11393|2021-09-29|      3 단계|  - 경남 전지역 3단계 (21....|
|경북|    8271|2021-09-29|      3 단계| - 경북 일부지역 3단계 (21...|
|전남|    3060|2021-09-29|      3 단계|  - 전남 전지역 3단계 (21....|
|전북|    4480|2021-09-29|      3 단계| - 전북 일부지역 3단계 (21...|
|충남|    9073|2021-09-29|      3 단계|  - 충남 전지역 3단계 (21....|
|충북|    6284|2021-09-29|      3 단계|  - 충북 전지역 3단계 (21....|
|강원|    6405|2021-09-29|      3 단계|- 춘천시, 원주시, 강릉시, ...|
|경기|   89246|2021-09-29|      4 단계|  - 경기 전지역 4단계 (21....|
|세종|    1225|2021-09-29|      3 단계|  - 세종 전지역 3단계 (21....|
|울산|    5138|2021-09-29|      3 단계|  - 울산 전지역 3단계 (21....|
|대전|    6949|2021-09-29|      3 단계|  - 대전 전지역 3단계 (21....|
|광주|    4957|2021-09-29|      3 단계|  - 광주 전지역 3단계 (21....|
|인천|   15654|2021-09-29|      4 단계| - 인천 일부지역

In [8]:
new_df = df_corona.toJSON().map(lambda x: json.loads(x)).collect()
new_df

Py4JJavaError: An error occurred while calling z:org.apache.spark.api.python.PythonRDD.collectAndServe.
: org.apache.spark.SparkException: Job aborted due to stage failure: Task 0 in stage 7.0 failed 1 times, most recent failure: Lost task 0.0 in stage 7.0 (TID 7) (ip-172-31-0-68.ap-northeast-2.compute.internal executor driver): org.apache.spark.api.python.PythonException: Traceback (most recent call last):
  File "/opt/spark/python/lib/pyspark.zip/pyspark/worker.py", line 604, in main
    process()
  File "/opt/spark/python/lib/pyspark.zip/pyspark/worker.py", line 596, in process
    serializer.dump_stream(out_iter, outfile)
  File "/opt/spark/python/lib/pyspark.zip/pyspark/serializers.py", line 259, in dump_stream
    vs = list(itertools.islice(iterator, batch))
  File "/opt/spark/python/lib/pyspark.zip/pyspark/util.py", line 73, in wrapper
    return f(*args, **kwargs)
  File "<ipython-input-8-e6984d20ea8f>", line 1, in <lambda>
NameError: name 'json' is not defined

	at org.apache.

In [ ]:
new_df = df_corona.toJSON().map(lambda x: json.loads(x)).collect()

for i in new_df:
    db.corona.insert_one(i)